In [15]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
from gensim.models import KeyedVectors
from nltk.corpus import brown
from nltk.corpus import treebank
from nltk.corpus import conll2000
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import random

In [21]:
treebank_corpus = treebank.tagged_sents(tagset='universal')
brown_corpus = brown.tagged_sents(tagset='universal')
conll_corpus = conll2000.tagged_sents(tagset='universal')
tagged_sentences = treebank_corpus + brown_corpus + conll_corpus

In [22]:
tagged_sentences[0]

[('Pierre', 'NOUN'),
 ('Vinken', 'NOUN'),
 (',', '.'),
 ('61', 'NUM'),
 ('years', 'NOUN'),
 ('old', 'ADJ'),
 (',', '.'),
 ('will', 'VERB'),
 ('join', 'VERB'),
 ('the', 'DET'),
 ('board', 'NOUN'),
 ('as', 'ADP'),
 ('a', 'DET'),
 ('nonexecutive', 'ADJ'),
 ('director', 'NOUN'),
 ('Nov.', 'NOUN'),
 ('29', 'NUM'),
 ('.', '.')]

In [23]:
X = [] # store input sequence
Y = [] # store output sequence

for sentence in tagged_sentences:
    X_sentence = []
    Y_sentence = []
    for entity in sentence:         
        X_sentence.append(entity[0])  # entity[0] contains the word
        Y_sentence.append(entity[1])  # entity[1] contains corresponding tag
        
    X.append(X_sentence)
    Y.append(Y_sentence)

In [24]:
X[0]

['Pierre',
 'Vinken',
 ',',
 '61',
 'years',
 'old',
 ',',
 'will',
 'join',
 'the',
 'board',
 'as',
 'a',
 'nonexecutive',
 'director',
 'Nov.',
 '29',
 '.']

In [47]:
Y[0]

['NOUN',
 'NOUN',
 '.',
 'NUM',
 'NOUN',
 'ADJ',
 '.',
 'VERB',
 'VERB',
 'DET',
 'NOUN',
 'ADP',
 'DET',
 'ADJ',
 'NOUN',
 'NOUN',
 'NUM',
 '.']

In [25]:
num_words = len(set([word.lower() for sentence in X for word in sentence]))
num_tags   = len(set([word.lower() for sentence in Y for word in sentence]))
print(num_words)
print(num_tags)

59448
12


In [26]:
unique_tags = list(set([word.lower() for sentence in Y for word in sentence]))
unique_tags_dict = {}
index = 1
for tag in unique_tags:
    unique_tags_dict[tag] = index 
    index += 1
print(unique_tags_dict)

{'adp': 1, 'prt': 2, 'adv': 3, 'det': 4, 'pron': 5, 'conj': 6, 'x': 7, 'noun': 8, 'verb': 9, 'num': 10, '.': 11, 'adj': 12}


In [27]:
unique_words = list(set([word.lower() for sentence in X for word in sentence]))
unique_words_dict = {}
index = 1
for word in unique_words:
    unique_words_dict[word] = index 
    index += 1
print(len(unique_words_dict))

59448


In [28]:
values = []
for i in unique_words_dict.keys():
    values.append(unique_words_dict[i])

print(max(values))
max_value_dict = max(values)

59448


In [29]:
def prepare_sequence_sentence(seq, to_ix):
    """Input: takes in a list of words, and a dictionary containing the index of the words
    Output: a tensor containing the indexes of the word"""
    idxs = [to_ix[w.lower()] for w in seq]
    random_index = random.randint(0,len(idxs)-1)
    idxs[random_index] = max_value_dict + 1
    return idxs

In [30]:
def prepare_sequence_tags(seq, to_ix):
    """Input: takes in a list of words, and a dictionary containing the index of the words
    Output: a tensor containing the indexes of the word"""
    idxs = [to_ix[w.lower()] for w in seq]
    return idxs

In [31]:
from keras_preprocessing.sequence import pad_sequences
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
def split(list_a, batch_size):

  for i in range(0, len(list_a), batch_size):
    yield list_a[i:i + batch_size]

cuda


In [32]:
batch_size = 8
X_batches = list(split(X, batch_size))
Y_batches = list(split(Y,batch_size))

In [33]:
X_batches_padded = []
Y_batches_padded = []
max_length_list = []

for b_s,b_t in zip(X_batches,Y_batches):
    max_seq_length = 0
    for sentence in b_s:
        if len(sentence) > max_seq_length:
            max_seq_length = len(sentence)
    
    sen_encoded = []
    tag_encoded = []
    for sentence,tags in zip(b_s,b_t):
        sen_encoded.append(prepare_sequence_sentence(sentence, unique_words_dict))
        tag_encoded.append(prepare_sequence_tags(tags, unique_tags_dict))
    
    X_batches_padded.append(pad_sequences(sen_encoded, maxlen=max_seq_length, padding="pre", truncating="post"))
    Y_batches_padded.append(pad_sequences(tag_encoded, maxlen=max_seq_length, padding="pre", truncating="post"))
    max_length_list.append(max_seq_length)

    

In [34]:
print(len(Y_batches_padded))
print(len(X_batches_padded))
print(len(max_length_list))


9026
9026
9026


In [35]:
print(len(Y_batches_padded[0]))
len(X_batches_padded[0])

8


8

In [36]:
print(Y_batches_padded[0])
print(X_batches_padded[0])
print(max_length_list[0])

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  8
   8 11 10  8 12 11  9  9  4  8  1  4 12  8  8 10 11]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  8  8  9  8  1  8  8 11  4  8  9  8 11]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  8  8 11 10  8 12  6 12  8  1
   8  8  8  8 11  9  9  7  4 12  8  1  4 12 12  8 11]
 [ 4  8  1  8  3  9  7  7  2  9  8  8  8  9  9  4 12  8  1  8  8  1  4  8
   1  8  9  7  2  5  3  1 10  8  1 11  8  9  7  7 11]
 [ 0  0  0  0  0  0  4  8  8 11  8 11  9  3 12  1  5  9  4  8 11  1  3 12
   8  2  5  9  8  4  7  9  2  8 12 11  8  9  7  7 11]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  8  8 11  4  8  1 12 12  8  8
   4  7  9  8  8 11  9  9  8  1  5  8  8  8  1 10 11]
 [ 0  0  0  0  1 12  8  9  9  7  3  1  4  8  1 11  4 12  8  9  1  8  2  8
   8  8  1  8 11  4  8 12  7  2  9 12  8  2  4  8 11]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
   0  0  0  0  0  4  8  8  9 1

In [37]:
X_final = []
Y_final = []

for index in range(len(X_batches_padded)):
    X_batch_tensor = torch.zeros((batch_size,max_length_list[index]),dtype = int).to(device= device)
    Y_batch_tensor = torch.zeros((batch_size,max_length_list[index]), dtype = int).to(device = device)

    count = 0
    for x, y in zip(X_batches_padded[index],Y_batches_padded[index]):
        X_batch_tensor[count] = torch.tensor(x).to(device = device)
        Y_batch_tensor[count] = torch.tensor(y).to(device =device)
        count += 1
    
    X_final.append(X_batch_tensor)
    Y_final.append(Y_batch_tensor)


In [38]:
print(len(X_final))
print(len(Y_final))
print(X_final[0].shape)
print(Y_final[0].shape)
print(X_final[0][0].shape)
print(Y_final[0][0].shape)
print(X_final[1].shape)
print(Y_final[1].shape)
print(X_final[0])
print(Y_final[0])

9026
9026
torch.Size([8, 41])
torch.Size([8, 41])
torch.Size([41])
torch.Size([41])
torch.Size([8, 38])
torch.Size([8, 38])
tensor([[    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0, 32437, 14675, 55964, 42148, 48419,  3949, 55964,
         59449, 12641, 38218, 39032, 53034, 48521, 58064,  4580, 50227,  1928,
         45147],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0, 35081, 14675,
          6610, 59088, 58469, 59353, 32350, 55964, 59449, 14694, 31562, 48546,
         45147],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0, 40508, 51290, 55964, 47716, 48419,  3949,
         20273, 51420, 59088, 58469, 57302, 43680, 1

In [18]:
# X_batches = []
# Y_batches = []
# batch_size = 8


# for sentence in X:
#     X_encoded.append(prepare_sequence(sentence, unique_words_dict))
# for tags in Y:
#     Y_encoded.append(prepare_sequence(tags, unique_tags_dict))

# MAX_SEQ_LENGTH = 100  # sequences greater than 100 in length will be truncated

# X_padded = pad_sequences(X_encoded, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")
# Y_padded = pad_sequences(Y_encoded, maxlen=MAX_SEQ_LENGTH, padding="pre", truncating="post")

In [19]:
# print(len(X_padded))
# X_padded[0]

In [20]:
# print(len(Y_encoded))
# print(Y_encoded)

In [39]:
EMBEDDING_SIZE  = 300  # each word in word2vec model is represented using a 300 dimensional vector
VOCABULARY_SIZE = num_words + 2

with open('./embedding_weights.pickle', 'rb') as file:
    embedding_weights = pickle.load(file)

print(embedding_weights.shape)

torch.Size([59450, 300])


In [40]:
def create_emb_layer(weights_matrix, non_trainable=False):

    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

In [41]:
class RNNTagger(nn.Module):
    def __init__(self, hidden_dim, target_size, batch_size):
        super(RNNTagger, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        
        self.word_embeddings, vocab_size, embedding_dim = create_emb_layer(embedding_weights, True)
        #self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first = True)
        self.hidden2tag = nn.Linear(hidden_dim, target_size)
        
    def forward(self, sentence):
        #print("REACHONG FORWARD")

        #Input shape: [batch_size,max_length in that batch]
        embeds = self.word_embeddings(sentence)
        #print("DONE embeds:", embeds.shape)  -- torch.Size([batch_Size, max_length_in_that_batch, embedding_dim])
  
        
        #input shape: [len(sentence),1,embedding_dim] (L,N,Hin​) when batch_first=False)
    
        rnn_out, hidden_state_out = self.rnn(embeds) 
        #print(rnn_out.shape)  -- torch.Size([8, 45, 64])
        #print(hidden_state_out.shape) --torch.Size([1, 8, 64])


        #input shape: -- torch.Size([8, 45, 64])
        tag_space = self.hidden2tag(rnn_out)
        #print("DONE LINEAR LAYER: ", tag_space.shape) --torch.Size([8, 45, 13])
        
        tag_scores = F.log_softmax(tag_space, dim=2)
        #print("DONE SOFTMAX:", tag_scores.shape) --torch.Size([8, 45, 13])
        
        return tag_scores

In [42]:
def train_loop(model,loss_function,optimizer,device,X,Y):
    train_length = len(X)
    epoch_train_loss = 0 
   
    model.train()
    for i in tqdm(range(train_length)):
        sentence_batch = X[i]
        tags_batch = Y[i]

        model.zero_grad()

        #print("Input shape:",  sentence_batch.shape) --torch.Size([batch_size, max_length_in_batch]) 
        tag_scores = model(sentence_batch)

       
        #print("model output shape: ", tag_scores.shape) -- torch.Size([8, 45, 13]) 
        #print("tags batch shape: ", tags_batch.shape) --torch.Size([8, 45])
        new1 = tag_scores.transpose(1,2)
        #print("new input shape:" , new1.shape) --torch.Size([8, 13, 45])

        #print("STARTING LOSS FUNCTION")
        loss = loss_function(new1 , tags_batch)
        #print("DONE WITH LOSS FUNCTION")
        #print(loss)
        epoch_train_loss += loss.item()
        loss.backward()
        optimizer.step()
    
    return model, epoch_train_loss/train_length

In [43]:
def validation_loop(model,loss_function,device,X,Y):
    val_length = len(X)
    epoch_val_loss = 0 

    for i in tqdm(range(val_length)):
        sentence_batch = X[i]
        tags_batch = Y[i]

        model.eval()

        tag_scores = model(sentence_batch)

        #CALL A FUNCTION WITH tag_scores and targets, GET PRECISION RECALL FScores        

        new1 = tag_scores.transpose(1,2) 
        loss = loss_function(new1, tags_batch)
        epoch_val_loss += loss.item()
          
     
    
    return epoch_val_loss/val_length

In [44]:
HIDDEN_DIM = 64
batch_size = 4
model =RNNTagger(HIDDEN_DIM, len(unique_tags_dict.keys())+1, batch_size)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device=device)

cuda


In [45]:
TEST_SIZE = 0.1
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=TEST_SIZE, random_state=4)

VALID_SIZE = 0.15
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=VALID_SIZE, random_state=4)

In [35]:
Y_train[1]

tensor([[ 0,  0,  0,  0,  0,  0,  7, 12, 12, 10,  7,  4, 12,  9, 12,  9, 12,  8,
         12, 11,  4, 12, 12, 10,  9,  7, 12, 10, 10,  1],
        [ 0,  0,  0,  0,  0,  0,  1,  9,  7,  4, 12,  1,  6, 10,  2, 10, 12,  1,
         11, 10, 12, 11,  7, 12,  1, 10,  7, 12, 12,  1],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  6, 10,  4, 11,
         12, 10,  9,  6, 10,  2, 10,  6, 12,  9,  1,  1],
        [ 1,  6, 10,  9,  6, 10,  6,  9,  7, 12,  9,  7, 12,  8, 10,  6,  7, 10,
          3, 12, 11, 10,  1,  1, 10,  3,  4, 12, 12,  1],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  2,  6, 10,  1,  1,  9,
          7, 12,  1,  3,  9,  7, 12, 10, 10,  1,  1,  1],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  7, 12, 12,  2, 10, 11, 12,  1],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0, 12, 12, 10,  7,  4, 12,  1],
        [ 0,  0,  0,  0,  0

In [36]:
X_train[1]

tensor([[    0,     0,     0,     0,     0,     0, 40706,  9997, 11719, 45610,
         20330, 14802, 48138, 20403, 40956, 20403, 57926, 28629, 11749, 36823,
         28446, 19444, 52177, 20809, 11132, 40706, 38278, 21834, 33386, 25158],
        [    0,     0,     0,     0,     0,     0, 27557, 45427, 40706, 23607,
         58336, 10989, 17660, 51960, 51269, 25802, 48913, 10120, 22995,  3927,
         52455, 22995, 40706, 34050, 10989, 21440, 20330, 26815, 40038, 25158],
        [    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0, 27557, 20487, 51735, 32999, 22995,  3476, 59002,
          4679, 30534, 51960, 10995, 49776,  1085, 25794, 46260, 25158, 10120],
        [27557, 20487, 51735, 41936, 30534,  5178, 17660, 20403, 20330,  8965,
         45427, 20330, 28018, 12198, 28309, 17660, 14836, 21834,  3812, 54173,
         22995,   716, 10989, 10120, 29244, 49756, 40481, 52177, 39465, 25158],
        [    0,     0,     0,     0,     0,     

In [37]:
print(len(X_train))
print(len(Y_train))

6904
6904


In [46]:
epochs = 5
for epoch in range(epochs):
    model , train_loss = train_loop(model,loss_function,optimizer,device,X_train,Y_train)
    val_loss = validation_loop(model,loss_function,device,X_validation,Y_validation)
    print("For epoch {}, training loss: {}, validation loss: {}".format(epoch, train_loss, val_loss))

100%|██████████| 1219/1219 [00:02<00:00, 516.53it/s]


For epoch 0, training loss: 0.47539491772215686, validation loss: 0.373599452425751


100%|██████████| 1219/1219 [00:02<00:00, 468.59it/s]


For epoch 1, training loss: 0.34039641293316114, validation loss: 0.3213791811718033


100%|██████████| 1219/1219 [00:02<00:00, 590.90it/s]


For epoch 2, training loss: 0.30288447106962857, validation loss: 0.297107857860582


100%|██████████| 1219/1219 [00:02<00:00, 425.18it/s]


For epoch 3, training loss: 0.28159749749853674, validation loss: 0.28230304639538945


100%|██████████| 1219/1219 [00:01<00:00, 763.28it/s]

For epoch 4, training loss: 0.2670571006022972, validation loss: 0.2720070388397298


DEMO CODE

In [53]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prana\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [56]:
def prepare_TESTsequence_sentence(seq, to_ix):
    values = []
    for i in to_ix.keys():
        values.append(to_ix[i])
    
    max_value_dict = max(values)
    
    idxs = []

    for w in seq:
        if w.lower() in to_ix.keys():
            idxs.append(to_ix[w.lower()])
        else:
            idxs.append(max_value_dict+1)
    return idxs

In [75]:
index_to_tags = {}
for tag in unique_tags_dict:
    index_to_tags[unique_tags_dict[tag]] = tag 
index_to_tags

{1: 'adp',
 2: 'prt',
 3: 'adv',
 4: 'det',
 5: 'pron',
 6: 'conj',
 7: 'x',
 8: 'noun',
 9: 'verb',
 10: 'num',
 11: '.',
 12: 'adj'}

In [77]:
from nltk.tokenize import word_tokenize
text = "bottle is 5 years old, non executive director of CSK"
tokenized_seq = word_tokenize(text)
print(tokenized_seq)

tokens= prepare_TESTsequence_sentence(tokenized_seq,unique_words_dict)
print(tokens) 

tokens = torch.tensor(tokens).to(device = device)
tokens = tokens.unsqueeze(0)
output = model(tokens)
output = output.squeeze(0)
tag_index = torch.argmax(output,dim=1)
#print(tag_index.shape)
#print(output.shape)
for predicted_tag,word in zip(tag_index,tokenized_seq):
    predicted_tag = predicted_tag.item()
    predicted_tag = index_to_tags[predicted_tag]
    print(word, predicted_tag)

    

['bottle', 'is', '5', 'years', 'old', ',', 'non', 'executive', 'director', 'of', 'CSK']
[56251, 6610, 58950, 48419, 3949, 55964, 14933, 22254, 4580, 58469, 59449]
bottle noun
is verb
5 num
years noun
old adj
, .
non noun
executive noun
director noun
of adp
CSK det


In [ ]:
#weight = torch.FloatTensor([[1, 2.3, 3], [4, 5.1, 6.3]])
embedding = nn.Embedding.from_pretrained(embedding_weights)
# Get embeddings for index 1
list1 = [5,6,8],
list1 = torch.tensor(list1)
list2 = [1,2,3]
list2 = torch.tensor(list2)
list3 = torch.zeros((2,3), dtype = int)
print(list1)
print(list2)
list3[0] = list1 
list3[1] = list2
print(list3)
a = embedding(list3)
a.shape -- batch_size,L_embedding_dim
#for i in list3:
#    print(i)
#    a = embedding(i)
#    print(a.shape)
#print(list1)
#input = torch.LongTensor([1])
#a = embedding(list3)
#a.shape

tensor([[5, 6, 8]])
tensor([1, 2, 3])
tensor([[5, 6, 8],
        [1, 2, 3]])


torch.Size([2, 3, 300])